### Decision Tree From Scratch

In [1]:
import numpy as np
from collections import Counter

#### 1. Implement the entropy given the target column

In [7]:
#First method(function) to calculates entropy for given target column

def ent(y):
    
    n = len(y) # number of data points 

    if n <= 1: # if n <= 1 it means that we have 1 or 0 data point and in that case entropy=0
        return 0

    counts = np.bincount(y) # count of unique values in y
    probs = counts[np.nonzero(counts)] / n
    n_classes = len(probs)

    if n_classes <= 1:
        return 0
    
    # calculate entropy
    return - np.sum(probs * np.log(probs)) / np.log(n_classes)


ent(np.array([1,0,1,0,1,0,0,0,0,0,1,0,1]))

0.961236604722876

In [8]:
# Second method(function) to calculate entropy  for given target column

def entropy(y):

    if len(y) <= 1: # if len(y) <= 1 it means that we have 1 or 0 data point and in that case entropy=0
        return 0

    counts = Counter()

    for d in y:
        counts[d] += 1

    entropy = 0

    probs = [float(c) / len(y) for c in counts.values()]
    
    for p in probs:
        if p > 0.:
            entropy -= p * np.log2(p)

    return entropy

entropy(np.array([1,0,1,0,1,0,0,0,0,0,1,0,1]))

0.9612366047228759

In [8]:
entropy(np.array([1,0,1,0,1,0,0,0,0,0,1,0,1]))

0.9612366047228759

#### 2. The below code defines the node class which we will use in tree creation.

In [9]:
class Node:

    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None


#### 3. Defining the Decision Tree.


### Modify the code where required.

In [10]:
class DecisionTree:

    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if (depth >= self.max_depth
                or n_labels == 1
                or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedily select the best split according to information gain
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        
        # grow the children that result from the split
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        """based on the entropy and the given split_thresh, calculate the information gain. """
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        # CODE HERE!
        e_l, e_r = entropy(y[left_idxs]),entropy(y[right_idxs]) # the entropy for left and right.
        child_entropy = n_l/n * e_l +  n_r /n * e_r # Child entropy


        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common
    
    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        """Using the traverse function created below find the prdicted value for each x."""                    
        return np.array([self._traverse_tree(x, self.root) for x in X])

#### Testing the algorithm


### After compliting the model try running it on the breast_cancer dataset using the code below and test if you are getting the same results.

In [11]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)
    
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)



print ("Accuracy:", acc)

Accuracy: 0.9122807017543859


Accuracy: 0.9122807017543859
